In [ ]:
import os
import pydicom
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import pandas as pd
from datetime import datetime
import hl7

# Globals
data_list = []
hl7_messages = {}

def read_dicom_files():
    file_paths = filedialog.askopenfilenames(title="Select DICOM files", filetypes=[("DICOM files", "*.dcm")])
    for file_path in file_paths:
        try:
            ds = pydicom.dcmread(file_path)
            name = getattr(ds, "PatientName", "Unknown")
            date = getattr(ds, "StudyDate", "00000000")
            ctdi = 0
            for elem in ds.iterall():
                if "CTDIvol" in str(elem.name):
                    try:
                        ctdi = float(elem.value)
                    except:
                        pass
            img = get_image(ds)
            display_image(img, name, date, ctdi)
            save_data(name, date, ctdi)
        except Exception as e:
            print("Failed to load:", e)

def get_image(ds):
    try:
        arr = ds.pixel_array
        image = Image.fromarray(arr)
        return image.resize((256, 256))
    except:
        return Image.new("L", (256, 256), color=128)

def display_image(img, name, date, ctdi):
    frame = tk.Frame(images_frame, bg="#ffffff", bd=2, relief="groove")
    frame.pack(side="left", padx=10, pady=10)
    imgtk = ImageTk.PhotoImage(img.convert("RGB"))
    lbl_img = tk.Label(frame, image=imgtk)
    lbl_img.image = imgtk
    lbl_img.pack()
    tk.Label(frame, text=f"Name: {name}", font=("Arial", 10), bg="#ffffff").pack()
    tk.Label(frame, text=f"Date: {format_date(date)}", font=("Arial", 10), bg="#ffffff").pack()
    tk.Label(frame, text=f"CTDIvol: {ctdi} mGy", font=("Arial", 10), bg="#ffffff", fg="green").pack()

def format_date(d):
    try:
        return datetime.strptime(d, "%Y%m%d").strftime("%d-%m-%Y")
    except:
        return "Unknown"

def save_data(name, date, ctdi):
    global data_list
    data_list.append({
        "PatientName": str(name),
        "StudyDate": date,
        "CTDIvol": ctdi
    })

def generate_csv_and_hl7():
    global data_list, hl7_messages
    df = pd.DataFrame(data_list)
    df['StudyDate'] = pd.to_datetime(df['StudyDate'], format="%Y%m%d", errors="coerce")
    df.dropna(inplace=True)

    output = []
    hl7_messages.clear()

    for patient in df['PatientName'].unique():
        patient_df = df[df['PatientName'] == patient].sort_values("StudyDate")
        start_date = patient_df['StudyDate'].min()
        end_date = start_date + pd.DateOffset(years=1)
        yearly_df = patient_df[(patient_df['StudyDate'] >= start_date) & (patient_df['StudyDate'] <= end_date)]
        total = yearly_df['CTDIvol'].sum()

        # Save patient HL7 message
        msg = hl7.parse(f"MSH|^~\\&|CT|HOSPITAL|RAD|HOSPITAL|{datetime.now().strftime('%Y%m%d')}||ORM^O01|MSG00001|P|2.3\rPID|||{patient}||\rOBX|1|NM|CTDIvol||{total}|mGy")
        hl7_messages[str(patient)] = msg

        output.append({
            "PatientName": patient,
            "StartDate": start_date.strftime("%Y-%m-%d"),
            "EndDate": end_date.strftime("%Y-%m-%d"),
            "Total_CTDIvol": total
        })

    result_df = pd.DataFrame(output)
    result_df.to_csv("rad.csv", index=False)
    print("Saved to rad.csv")

def search_hl7():
    name = search_entry.get().strip()
    if name in hl7_messages:
        text.delete(1.0, tk.END)
        text.insert(tk.END, str(hl7_messages[name]))
    else:
        text.delete(1.0, tk.END)
        text.insert(tk.END, "No HL7 message found for this patient.")

# GUI setup
root = tk.Tk()
root.title("Smart DICOM Viewer with CTDIvol")
root.geometry("1200x700")

bg_img = Image.open(r"background-medical.jpg").resize((1200, 700))
bg_photo = ImageTk.PhotoImage(bg_img)
bg_label = tk.Label(root, image=bg_photo)
bg_label.place(x=0, y=0, relwidth=1, relheight=1)

btn = tk.Button(root, text="📂 Select DICOM Files", command=read_dicom_files, font=("Arial", 14), bg="green", fg="white")
btn.pack(pady=10)

images_frame = tk.Frame(root, bg="white")
images_frame.pack(fill="both", expand=True)

bottom_frame = tk.Frame(root, bg="#f0f0f0")
bottom_frame.pack(fill="x", pady=5)

tk.Button(bottom_frame, text="📄 Save CSV + HL7", command=generate_csv_and_hl7, bg="blue", fg="white").pack(side="left", padx=10)
search_entry = tk.Entry(bottom_frame)
search_entry.pack(side="left")
tk.Button(bottom_frame, text="🔍 Find HL7 by Patient", command=search_hl7).pack(side="left", padx=5)

text = tk.Text(bottom_frame, height=5)
text.pack(side="left", fill="x", expand=True, padx=10)

root.mainloop()


In [ ]:
import os
import pydicom
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import pandas as pd
from datetime import datetime
import hl7

# Globals
data_list = []
hl7_messages = {}

def read_dicom_files():
    file_paths = filedialog.askopenfilenames(title="Select DICOM files", filetypes=[("DICOM files", "*.dcm")])
    for file_path in file_paths:
        try:
            ds = pydicom.dcmread(file_path)
            name = getattr(ds, "PatientName", "Unknown")
            date = getattr(ds, "StudyDate", "00000000")
            ctdi = 0
            for elem in ds.iterall():
                if "CTDIvol" in str(elem.name):
                    try:
                        ctdi = float(elem.value)
                    except:
                        pass
            img = get_image(ds)
            display_image(img, name, date, ctdi)
            save_data(name, date, ctdi)
        except Exception as e:
            print("Failed to load:", e)

def get_image(ds):
    try:
        arr = ds.pixel_array
        image = Image.fromarray(arr)
        return image.resize((256, 256))
    except:
        return Image.new("L", (256, 256), color=128)

def display_image(img, name, date, ctdi):
    frame = tk.Frame(images_frame, bg="#ffffff", bd=2, relief="groove")
    frame.pack(side="left", padx=10, pady=10)
    imgtk = ImageTk.PhotoImage(img.convert("RGB"))
    lbl_img = tk.Label(frame, image=imgtk)
    lbl_img.image = imgtk
    lbl_img.pack()
    tk.Label(frame, text=f"Name: {name}", font=("Arial", 10), bg="#ffffff").pack()
    tk.Label(frame, text=f"Date: {format_date(date)}", font=("Arial", 10), bg="#ffffff").pack()
    tk.Label(frame, text=f"CTDIvol: {ctdi} mGy", font=("Arial", 10), bg="#ffffff", fg="green").pack()

def format_date(d):
    try:
        return datetime.strptime(d, "%Y%m%d").strftime("%d-%m-%Y")
    except:
        return "Unknown"

def save_data(name, date, ctdi):
    global data_list
    data_list.append({
        "PatientName": str(name),
        "StudyDate": date,
        "CTDIvol": ctdi
    })

def generate_csv_and_hl7():
    global data_list, hl7_messages
    df = pd.DataFrame(data_list)
    df['StudyDate'] = pd.to_datetime(df['StudyDate'], format="%Y%m%d", errors="coerce")
    df.dropna(inplace=True)

    output = []
    hl7_messages.clear()

    for patient in df['PatientName'].unique():
        patient_df = df[df['PatientName'] == patient].sort_values("StudyDate")
        start_date = patient_df['StudyDate'].min()
        end_date = start_date + pd.DateOffset(years=1)
        yearly_df = patient_df[(patient_df['StudyDate'] >= start_date) & (patient_df['StudyDate'] <= end_date)]
        total = yearly_df['CTDIvol'].sum()

        # Save patient HL7 message
        msg = hl7.parse(f"MSH|^~\\&|CT|HOSPITAL|RAD|HOSPITAL|{datetime.now().strftime('%Y%m%d')}||ORM^O01|MSG00001|P|2.3\rPID|||{patient}||\rOBX|1|NM|CTDIvol||{total}|mGy")
        hl7_messages[str(patient)] = msg

        output.append({
            "PatientName": patient,
            "StartDate": start_date.strftime("%Y-%m-%d"),
            "EndDate": end_date.strftime("%Y-%m-%d"),
            "Total_CTDIvol": total
        })

    result_df = pd.DataFrame(output)
    result_df.to_csv("rad.csv", index=False)
    print("Saved to rad.csv")

def search_hl7():
    name = search_entry.get().strip()
    if name in hl7_messages:
        text.delete(1.0, tk.END)
        text.insert(tk.END, str(hl7_messages[name]))
    else:
        text.delete(1.0, tk.END)
        text.insert(tk.END, "No HL7 message found for this patient.")

# GUI setup
root = tk.Tk()
root.title("Smart DICOM Viewer with CTDIvol")
root.geometry("1200x700")

bg_img = Image.open(r"background-medical.jpg").resize((1200, 700))
bg_photo = ImageTk.PhotoImage(bg_img)
bg_label = tk.Label(root, image=bg_photo)
bg_label.place(x=0, y=0, relwidth=1, relheight=1)

btn = tk.Button(root, text="📂 Select DICOM Files", command=read_dicom_files, font=("Arial", 14), bg="green", fg="white")
btn.pack(pady=10)

images_frame = tk.Frame(root, bg="white")
images_frame.pack(fill="both", expand=True)

bottom_frame = tk.Frame(root, bg="#f0f0f0")
bottom_frame.pack(fill="x", pady=5)

tk.Button(bottom_frame, text="📄 Save CSV + HL7", command=generate_csv_and_hl7, bg="blue", fg="white").pack(side="left", padx=10)
search_entry = tk.Entry(bottom_frame)
search_entry.pack(side="left")
tk.Button(bottom_frame, text="🔍 Find HL7 by Patient", command=search_hl7).pack(side="left", padx=5)

text = tk.Text(bottom_frame, height=5)
text.pack(side="left", fill="x", expand=True, padx=10)

root.mainloop()


In [ ]:
import os
import pydicom
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import pandas as pd
from datetime import datetime
import hl7

# Globals
data_list = []
hl7_messages = {}

def read_dicom_files():
    file_paths = filedialog.askopenfilenames(title="Select DICOM files", filetypes=[("DICOM files", "*.dcm")])
    for file_path in file_paths:
        try:
            ds = pydicom.dcmread(file_path)
            name = getattr(ds, "PatientName", "Unknown")
            date = getattr(ds, "StudyDate", "00000000")
            ctdi = 0
            for elem in ds.iterall():
                if "CTDIvol" in str(elem.name):
                    try:
                        ctdi = float(elem.value)
                    except:
                        pass
            img = get_image(ds)
            display_image(img, name, date, ctdi)
            save_data(name, date, ctdi)
        except Exception as e:
            print("Failed to load:", e)

def get_image(ds):
    try:
        arr = ds.pixel_array
        image = Image.fromarray(arr)
        return image.resize((256, 256))
    except:
        return Image.new("L", (256, 256), color=128)

def display_image(img, name, date, ctdi):
    frame = tk.Frame(images_frame, bg="#ffffff", bd=2, relief="groove")
    frame.pack(side="left", padx=10, pady=10)
    imgtk = ImageTk.PhotoImage(img.convert("RGB"))
    lbl_img = tk.Label(frame, image=imgtk)
    lbl_img.image = imgtk
    lbl_img.pack()
    tk.Label(frame, text=f"Name: {name}", font=("Arial", 10), bg="#ffffff").pack()
    tk.Label(frame, text=f"Date: {format_date(date)}", font=("Arial", 10), bg="#ffffff").pack()
    tk.Label(frame, text=f"CTDIvol: {ctdi} mGy", font=("Arial", 10), bg="#ffffff", fg="green").pack()

def format_date(d):
    try:
        return datetime.strptime(d, "%Y%m%d").strftime("%d-%m-%Y")
    except:
        return "Unknown"

def save_data(name, date, ctdi):
    global data_list
    data_list.append({
        "PatientName": str(name),
        "StudyDate": date,
        "CTDIvol": ctdi
    })

def generate_csv_and_hl7():
    global data_list, hl7_messages
    df = pd.DataFrame(data_list)
    df['StudyDate'] = pd.to_datetime(df['StudyDate'], format="%Y%m%d", errors="coerce")
    df.dropna(inplace=True)

    output = []
    hl7_messages.clear()

    for patient in df['PatientName'].unique():
        patient_df = df[df['PatientName'] == patient].sort_values("StudyDate")
        start_date = patient_df['StudyDate'].min()
        end_date = start_date + pd.DateOffset(years=1)
        yearly_df = patient_df[(patient_df['StudyDate'] >= start_date) & (patient_df['StudyDate'] <= end_date)]
        total = yearly_df['CTDIvol'].sum()

        # Save patient HL7 message
        msg = hl7.parse(f"MSH|^~\\&|CT|HOSPITAL|RAD|HOSPITAL|{datetime.now().strftime('%Y%m%d')}||ORM^O01|MSG00001|P|2.3\rPID|||{patient}||\rOBX|1|NM|CTDIvol||{total}|mGy")
        hl7_messages[str(patient)] = msg

        output.append({
            "PatientName": patient,
            "StartDate": start_date.strftime("%Y-%m-%d"),
            "EndDate": end_date.strftime("%Y-%m-%d"),
            "Total_CTDIvol": total
        })

    result_df = pd.DataFrame(output)
    result_df.to_csv("rad.csv", index=False)
    print("Saved to rad.csv")

def search_hl7():
    name = search_entry.get().strip()
    if name in hl7_messages:
        text.delete(1.0, tk.END)
        text.insert(tk.END, str(hl7_messages[name]))
    else:
        text.delete(1.0, tk.END)
        text.insert(tk.END, "No HL7 message found for this patient.")

# GUI setup
root = tk.Tk()
root.title("Smart DICOM Viewer with CTDIvol")
root.geometry("1200x700")

bg_img = Image.open(r"background-medical.jpg").resize((1200, 700))
bg_photo = ImageTk.PhotoImage(bg_img)
bg_label = tk.Label(root, image=bg_photo)
bg_label.place(x=0, y=0, relwidth=1, relheight=1)

btn = tk.Button(root, text="📂 Select DICOM Files", command=read_dicom_files, font=("Arial", 14), bg="green", fg="white")
btn.pack(pady=10)

images_frame = tk.Frame(root, bg="white")
images_frame.pack(fill="both", expand=True)

bottom_frame = tk.Frame(root, bg="#f0f0f0")
bottom_frame.pack(fill="x", pady=5)

tk.Button(bottom_frame, text="📄 Save CSV + HL7", command=generate_csv_and_hl7, bg="blue", fg="white").pack(side="left", padx=10)
search_entry = tk.Entry(bottom_frame)
search_entry.pack(side="left")
tk.Button(bottom_frame, text="🔍 Find HL7 by Patient", command=search_hl7).pack(side="left", padx=5)

text = tk.Text(bottom_frame, height=5)
text.pack(side="left", fill="x", expand=True, padx=10)

root.mainloop()
